In [ ]:
#Simulation of the setup with the rotating galvo mirror

# IMPORTS
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.stats import poisson
from scipy.optimize import curve_fit

# CONSTANTS

QD_constants = True
Custome_constants = False

if QD_constants:
# Probability of absorbing photons based on Poisson statistics
    exp_abs = 0.10
    zero_chance = poisson.pmf(0, exp_abs)  # Probability of absorbing 0 photons
    one_chance = poisson.pmf(1, exp_abs)  # Probability of absorbing 1 photon

# Laser and detection system constants
    freq_laser = 2.5  # Frequency of the laser in MHz
    interval = (1/freq_laser)*1e6 #Calculates the interval between pulses in ps
    mean_x = 633  # Mean wavelength for X state (nm)
    standard_deviation_x = 10  # Standard deviation for X state (nm)
    mean_bx = 629  # Mean wavelength for BX state (nm)
    standard_deviation_bx = 10  # Standard deviation for BX state (nm)

# State lifetimes in picoseconds (ps)
    mean_life_x = int(24 * 1e3)  # Mean lifetime of X state
    rad = 0.105  # Ratio of BX states decaying radiatively
    mean_life_rd_bx = int((1/(2*rad))*1e3) # Mean lifetime of BX state
    mean_life_bx = 1 / ((1 / mean_life_rd_bx)/rad)
    mean_life_a_bx = 1 / ((1 / mean_life_bx) * ((1 - rad) / rad))  # Non-radiative BX lifetime

# Wavelength detection range
    scanning_min_input = 590  # Minimum wavelength (nm)
    scanning_max_input = 670  # Maximum wavelength (nm)
    freq_mirror = 0.5 # Mirror rotation frequency (Hz)
    freq_mirror = int(1/(freq_mirror)*1e12) #Chances it into the rigth unites

# System parameters
    system_eff = 0.2  # Overall detection efficiency {Not given in the paper so estimated}
    g2_times = 4  # Number of detection events considered in g(2) calculation
    down_time = 25e3  # Dead time of a pixel (ps)
    darkrate = 5 #Amount of dark counts per pixel per second

# Simulation configurations
    run_time_list = [600] # Simulation runtimes (s) {Paper measures between 30-60 min}
    delta_lambda_list = [13] # Resolution (nm)	
    repeats = 1 # Number of simulation repetitions

if Custome_constants:
# Probability of absorbing photons based on Poisson statistics
    exp_abs = 0.1
    zero_chance = poisson.pmf(0, exp_abs)  # Probability of absorbing 0 photons
    one_chance = poisson.pmf(1, exp_abs)  # Probability of absorbing 1 photon

# Laser and detection system constants
    freq_laser = 1  # Frequency of the laser in MHz
    interval = (1/freq_laser)*1e6 #Calculates the interval between pulses in ps
    mean_x = 620  # Mean wavelength for X state (nm)
    standard_deviation_x = 10  # Standard deviation for X state (nm)
    mean_bx = 623  # Mean wavelength for BX state (nm)
    standard_deviation_bx = 10  # Standard deviation for BX state (nm)

# State lifetimes in picoseconds (ps)
    mean_life_x = int(20 * 1e3)  # Mean lifetime of X state
    mean_life_bx = int(2 * 1e3)  # Mean lifetime of BX state
    rad = 0.1  # Ratio of BX states decaying radiatively
    mean_life_a_bx = 1 / ((1 / mean_life_bx) * ((1 - rad) / rad))  # Non-radiative BX lifetime

# Wavelength detection range
    scanning_min_input = 580  # Minimum wavelength (nm)
    scanning_max_input = 660  # Maximum wavelength (nm)
    freq_mirror = 0.5 # Mirror rotation frequency (Hz)
    freq_mirror = int(1/(freq_mirror)*1e12) #Chances it into the rigth unites

# System parameters
    system_eff = 0.2  # Overall detection efficiency
    g2_times = 4  # Number of detection events considered in g(2) calculation
    down_time = 25e3  # Dead time of a pixel (ps)
    darkrate = 10 #Amount of dark counts per pixel per second

# Simulation configurations
    run_time_list = [300]  # Simulation runtimes (s)
    delta_lambda_list = [6] # Resolution (nm)
    repeats = 1 # Number of simulation repetitions

Darkcounts_Galvo = True
afterpulsingfilter = True
plotsshow = True

# DEFINITIONS

def decay(mean_lifetime):
    """
    Generate a random lifetime using an exponential distribution.
    :param mean_lifetime: Average lifetime of the state (ps)
    :return: Random lifetime (ps)
    """
    return int(np.random.exponential(mean_lifetime))

def wavelength(mean, sd):
    """
    Generate a random wavelength using a Gaussian distribution.
    :param mean: Mean wavelength (nm)
    :param sd: Standard deviation of the wavelength (nm)
    :return: Random wavelength (nm)
    """
    return np.random.normal(mean, sd)

def mirrorpos(t):
    """
    Calculate the mirror position and corresponding center wavelength at time t.
    :param t: Time (s)
    :return: Center wavelength at time t (nm)
    """
    return scanning_min_input + ((2*(scanning_max_input-scanning_min_input))/freq_mirror)*abs((((t)) % freq_mirror)-(freq_mirror/2))

def g2(time_total):
    """
    Compute g(2) by calculating time differences between detections.
    :param time_total: Array of detections as [time, detector]
    :return: Time differences for g(2) calculation
    """
    delay1 = []
    for i in range(g2_times):
        if len(delay1) == 0 and i != 0:
            delay1 = time_total[i:] - time_total[:-i]
            delay1 = delay1[(abs(delay1[:, 0]) <= 1.5 * interval) * (delay1[:, 1] != 0)]
            delay1 = np.concatenate((delay1, time_total[:-i] - time_total[i:]))
        elif i != 0:
            delay1 = np.concatenate((delay1, time_total[i:] - time_total[:-i]))
            delay1 = delay1[(abs(delay1[:, 0]) <= 1.5 * interval) * (delay1[:, 1] != 0)]
    return delay1[:, 0]

def double_exp_func(x, a, b, d, e, c):
    """
    Double exponential function for fitting decay curves.
    :param x: Time (ps)
    :param a, b, d, e, c: Fit parameters
    :return: Function value
    """
    return a * np.exp(-(1/b) * x) + d * np.exp(-(1/e) * x) + c

def exp_func(x, a, b, c):
    """
    Single exponential function for fitting decay curves.
    :param x: Time (ps)
    :param a, b, c: Fit parameters
    :return: Function value
    """
    return a * np.exp(-(1/b) * x) + c

def normalize(arr):
    """
    Normalize an array to range [0, 1].
    :param arr: Input array
    :return: Normalized array
    """
    return (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

def gaussian(x, c, mu, sigma, d):
    """
    Gaussian function for fitting spectral data.
    :param x: Wavelength (nm)
    :param c, mu, sigma, d: Fit parameters
    :return: Function value
    """
    return abs(c * np.exp(-(x - mu)**2 / (2 * sigma**2))) + d


def g_2_fit(x, a, b, d, e, f):
    """
    Model function for fitting g(2) curves.
    :param x: Time difference (ps)
    :param a, b, d, e, f: Fit parameters
    :return: Function value
    """
    return a * np.exp(-1 * abs(x) * e) + \
           b * np.exp(-1 * abs(x - f) * e) + \
           b * np.exp(-1 * abs(x + f) * e) + d

# ----------Galvo Loop----------

# Lists where the output is put in
x_spec_fit_g, x_spec_err_g, bx_spec_fit_g, bx_spec_err_g, bx_decay_fit_g, bx_decay_err_g, x_decay_fit_g, x_decay_err_g = [], [], [], [], [], [], [], []
x_spec_time_g, x_spec_lambda_g, bx_spec_time_g, bx_spec_lambda_g, bx_decay_time_g, bx_decay_lambda_g, x_decay_time_g, x_decay_lambda_g = [], [], [], [], [], [], [], []
darkscounted, bx_count_list, x_count_list = [], [], []
g_2, g_2_err = [], []

for i in range(len(run_time_list)):

# Gets run time and calculates necessary constants
    run_time_in_s = run_time_list[i]
    run_time = int(run_time_in_s*1e12) #run_time of the entire experiment in ps         
    pulses = int(run_time/interval) #Calculates the total amount of pulses in the entire experiment in ps

    for j in range(len(delta_lambda_list)):
        delta_lambda_apd = delta_lambda_list[j]/2

        def gaussian_box(x, c, mu, sigma, d):
            delta_lambda_apd1 = delta_lambda_list[j]
            res = c * (((1/2)*erf((x + (delta_lambda_apd1/2) - mu ) / (np.sqrt(2) * sigma))
                        - (1/2)*erf((x - (delta_lambda_apd1/2)  - mu ) / (np.sqrt(2) * sigma)))) + d
            return res

        scanning_min = scanning_min_input-delta_lambda_apd/2 #nm
        scanning_max = scanning_max_input+delta_lambda_apd/2 #m

        for k in range(repeats):            
            print(i+1,"/",len(run_time_list),",",j+1,"/",len(delta_lambda_list),",",k+1,"/",repeats)

# Determines the amount of X and BX detections and defines the decayrates
            number_x = int(np.random.normal(((pulses)*(1-zero_chance))))
            number_bx = int(np.random.normal(((pulses)*(1-zero_chance-one_chance)))) 
            number_a_bx = int(number_bx*(1-rad))

# Creates a lists with lifetimes for the X where no BX is detected, and a list of zeros for the BX. It also generates a little less of these becayse of system eff. this will later be forced on the X with a BX as well
            photon_bx_arr_1 = (np.zeros(int((number_x-number_bx)*system_eff))).astype(np.int8)
            photon_x_arr_1= (-mean_life_x*np.log(np.random.uniform(0,1,(int((number_x-number_bx)*system_eff))))).astype(np.int64)
            detector_1_x = np.random.randint(1,3,int(((number_x-number_bx)*system_eff))).astype(np.int8) #generates a random list of 1 or 2 to determine the destination of the photons
            detector_1_bx = (np.zeros(int((number_x-number_bx)*system_eff))).astype(np.int8)

# Creates 2 lists with lifetimes for the X and BX which will decay radiative
            photon_bx_arr_2 = (-mean_life_bx*np.log(np.random.uniform(0,1,number_bx-number_a_bx))).astype(np.int32)
            photon_x_arr_2 =(-mean_life_x*np.log(np.random.uniform(0,1,number_bx-number_a_bx))).astype(np.int64)
            detector_2_x = np.random.randint(1,3,int((number_bx-number_a_bx))).astype(np.int8) #generates a random list of 1 or 2 to determine the destination of the photons
            detector_2_bx = np.random.randint(1,3,int((number_bx-number_a_bx))).astype(np.int8) #generates a random list of 1 or 2 to determine the destination of the photons

# Creates 2 lists with lifetimes for the X and BX which will decay via Auger and system_eff gets applied
            photon_bx_arr_3 = (-mean_life_a_bx*np.log(np.random.uniform(0,1,int(number_a_bx*system_eff)))).astype(np.int32)
            photon_x_arr_3 =(-mean_life_x*np.log(np.random.uniform(0,1,int(number_a_bx*system_eff)))).astype(np.int64)
            detector_3_x = np.random.randint(1,3,int(number_a_bx*system_eff)).astype(np.int8) #generates a random list of 1 or 2 to determine the destination of the photons
            detector_3_bx = np.random.randint(1,3,int(number_a_bx*system_eff)).astype(np.int8) #generates a random list of 1 or 2 to determine the destination of the photons

# All lists get added togheter so we get one array with all the data. (BX_lifetime,X_lifetime,Detectors)
            photon_arr_1 = np.vstack((photon_bx_arr_1,photon_x_arr_1,detector_1_bx, detector_1_x)).T
            photon_arr_2 = np.vstack((photon_bx_arr_2,photon_x_arr_2,detector_2_bx, detector_2_x)).T
            photon_arr_3 = np.vstack((photon_bx_arr_3,photon_x_arr_3,detector_3_bx, detector_3_x)).T
            photon_arr = np.vstack((photon_arr_1,photon_arr_3,photon_arr_2))

# creates a list of pulse times and choices random times out of a list to choose the arrival times of the photons
            rng = np.random.default_rng()
            numbertimes = 60000
            time_total_parts = []

            photons_per_part = int((len(photon_arr) + numbertimes) / numbertimes)

            for l in range(numbertimes):

                time_total_part = (rng.choice(np.arange(int(l * pulses / numbertimes), int((l + 1) * pulses / numbertimes)), size=photons_per_part, replace=False) * interval).astype(np.int64)
                time_total_parts.append(time_total_part)

            time_total = np.concatenate(time_total_parts)[:len(photon_arr)]

# Makes is so that the BX lifetime gets added to the X lifetime, where they are both within 1 interval
            photon_arr[:, :2] = np.cumsum(photon_arr[:, :2], axis=1)

# creates a mask, so where there are liftimes on the list a random time is added. But in such a way that the BX is always with a X.
            macro_ = np.vstack((time_total, time_total)).T

            mask = photon_arr[:,:2] != 0

            photon_arr[:,:2][mask] += macro_[mask]

# System_eff for X with BX

# Here some of the entries for the X with a BX and BX lists gets destroyed due to system eff.
            remain_numb = int((number_bx-number_a_bx)*system_eff)
            delete_numb = int((number_bx-number_a_bx)-remain_numb)
            system_eff_arr = [1]*remain_numb + [0]*delete_numb
            system_eff_arr = np.array(system_eff_arr).astype(int)
            np.random.shuffle(system_eff_arr)
            photon_arr[int((len(photon_arr) - len(photon_arr_2))):,0] = photon_arr[int(len(photon_arr) - len(photon_arr_2)):,0] * system_eff_arr
            photon_arr[int(len(photon_arr) - len(photon_arr_2)):,2] = photon_arr[int(len(photon_arr) - len(photon_arr_2)):,2] * system_eff_arr
            np.random.shuffle(system_eff_arr)
            photon_arr[int((len(photon_arr) - len(photon_arr_2))):,1] = photon_arr[int(len(photon_arr) - len(photon_arr_2)):,1] * system_eff_arr
            photon_arr[int(((len(photon_arr) - len(photon_arr_2)))):,3] = photon_arr[int(len(photon_arr) - len(photon_arr_2)):,3] * system_eff_arr

# Rad or Non-rad for BX

            photon_arr[len(photon_arr_1):len(photon_arr_1)+len(photon_arr_3),0] = len(photon_arr_3)*[0]

# Here the list gets sorted in APD1 and APD2. with a filter for deadtime and the Galvo mirror

# BXs
            total_bx_photon_arr = np.vstack((photon_arr[:,0],photon_arr[:,2])).astype(np.int64).T
            total_bx_photon_arr = total_bx_photon_arr[np.where(total_bx_photon_arr[:,0] != 0)]

            total_bx_photon_arr_1 = total_bx_photon_arr[np.where(total_bx_photon_arr[:,1] == 1)]
            total_bx_photon_arr_1 = total_bx_photon_arr_1[:,0]

            total_bx_photon_arr_2 = total_bx_photon_arr[np.where(total_bx_photon_arr[:,1] != 1)]
            total_bx_photon_arr_2 = total_bx_photon_arr_2[:,0]
            bx_wavelengths = np.random.normal(mean_bx, standard_deviation_bx, len(total_bx_photon_arr_2))
            total_bx_photon_arr_2 = np.vstack((total_bx_photon_arr_2,bx_wavelengths)).T

# Xs
            total_x_photon_arr = np.vstack((photon_arr[:,1],photon_arr[:,3])).astype(np.int64).T

            total_x_photon_arr_1 = total_x_photon_arr[np.where(total_x_photon_arr[:,1] == 1)]
            total_x_photon_arr_1 = total_x_photon_arr_1[:,0]

            total_x_photon_arr_2 = total_x_photon_arr[np.where(total_x_photon_arr[:,1] == 2)]
            total_x_photon_arr_2 = total_x_photon_arr_2[:,0]
            x_wavelengths = np.random.normal(mean_x, standard_deviation_x, len(total_x_photon_arr_2))
            total_x_photon_arr_2 = np.vstack((total_x_photon_arr_2,x_wavelengths)).T

# The lists get made into one APD1 list and one APD2 list
            total_photon_arr_1 = np.concatenate((total_bx_photon_arr_1, total_x_photon_arr_1))
            total_photon_arr_2 = np.concatenate((total_bx_photon_arr_2, total_x_photon_arr_2))

# The entries which dont make it to APD2 because of the Galvo mirror gets filtered out
            total_photon_arr_2= total_photon_arr_2[np.argsort(total_photon_arr_2[:,0])]
            time_2 = total_photon_arr_2[:,0] 
            wave_2 = total_photon_arr_2[:,1]  
            time_2 = mirrorpos(time_2)

            diff = abs((time_2-wave_2))

            mask = diff >= delta_lambda_apd
            total_photon_arr_2 = total_photon_arr_2[~mask]
            time_2 = time_2[~mask]
            total_photon_arr_2[:,1] = time_2

# Darkcounts
# Here a list of darkcounts is generates on random sensors and made into one array with each other
            if Darkcounts_Galvo:
                darks_1 =  np.random.uniform(0,int(run_time),int(run_time_in_s*darkrate)).astype(np.int64) #generates uniform numbers over the measurment limits
                darks_2 =  np.random.uniform(0,int(run_time),int(run_time_in_s*darkrate)).astype(np.int64) #generates uniform numbers over the measurment limits

                detect_darkcounts = np.random.randint(scanning_min_input,scanning_max_input+1,run_time_in_s*darkrate)
                total_darkcount_arr_2 = np.vstack((darks_1, detect_darkcounts)).T

                total_photon_arr_1 = np.concatenate((total_photon_arr_1, darks_2))
                total_photon_arr_1= total_photon_arr_1[np.argsort(total_photon_arr_1)]

                total_photon_arr_2 = np.concatenate((total_photon_arr_2, total_darkcount_arr_2))
                total_photon_arr_2= total_photon_arr_2[np.argsort(total_photon_arr_2[:, 0])]

# Applies deadtime filter

            total_photon_arr_1 = total_photon_arr_1[~((np.concatenate(([False],((np.diff(total_photon_arr_1)) < down_time)))))] 
            total_photon_arr_2 = total_photon_arr_2[~((np.concatenate(([False],((np.diff(total_photon_arr_2[:,0])) < down_time))))*(np.concatenate(([True],((np.diff(total_photon_arr_2[:,1])) == 0)))))] 

# Applies a afterpulsing filter removing if detection on the same pixel come shortly after each other
            if afterpulsingfilter:
                total_photon_arr_1 = total_photon_arr_1[~(np.concatenate(([False],((np.diff(total_photon_arr_1) < interval)*(np.diff(np.mod(total_photon_arr_1,interval))>0)))))]
                total_photon_arr_2 = total_photon_arr_2[~(np.concatenate(([False],((np.diff(total_photon_arr_2[:,0]) < interval)*(np.diff(np.mod(total_photon_arr_2[:,0],interval))>0)))))]

            total_photon_arr_1_wave = np.vstack((total_photon_arr_1,np.zeros(len(total_photon_arr_1)))).T
            total_photon_arr_wave = np.concatenate((total_photon_arr_1_wave,total_photon_arr_2))
            total_photon_arr_wave = total_photon_arr_wave[np.argsort(total_photon_arr_wave[:,0])]

# Adds apd 1 and 2 toghether
            total_photon_arr_1_ = np.vstack((total_photon_arr_1,len(total_photon_arr_1)*[1])).T
            total_photon_arr_2_ = np.vstack((total_photon_arr_2[:,0],len(total_photon_arr_2)*[2])).T
            total_photon_arr = np.concatenate((total_photon_arr_1_,total_photon_arr_2_))
            total_photon_arr= total_photon_arr[np.argsort(total_photon_arr[:,0])]

# Calculates and plots the X spectrum

            total_detect_arr = total_photon_arr_wave[total_photon_arr_wave[:,1] > 0][:,1]

            bins = np.arange(scanning_min_input,scanning_max_input+2, 2)
            emission_hist, bin_edges = np.histogram(total_detect_arr, bins=bins)

            bins = (bin_edges[:-1]+bin_edges[1:])/2

# best fit of data
            popt, pcov = curve_fit(gaussian_box, bins, normalize(emission_hist), (1.0, mean_x, standard_deviation_x,1), maxfev=100000, bounds=((0, 0, 0,0), (np.inf, np.inf, np.inf, np.inf)), absolute_sigma=False)
            p_sigma_x_galvo = np.sqrt(np.diag(pcov))

            x_spec_lambda_g.append(2 * delta_lambda_apd)
            x_spec_time_g.append(run_time_in_s)
            x_spec_fit_g.append((popt[1]))
            x_spec_err_g.append((p_sigma_x_galvo[1]))

            if plotsshow:
                plt.figure(dpi=300)  # Increase figure size and DPI for higher quality, make it square
                plt.plot(bins, gaussian_box(bins, *popt), c="darkorange", linewidth=2, label="Spectrum Fit")
                plt.scatter(bins, normalize(emission_hist), c="darkblue", s=30, label="Spectrum Measurements", zorder=5)
                plt.legend(loc="lower right")
                plt.xlabel("Wavelength (nm)")
                plt.ylabel("Intensity (norm.)")
                plt.ylim(0,)
                plt.xlim(scanning_min_input, scanning_max_input)
                plt.minorticks_on()
                plt.xticks()
                plt.yticks()
                plt.show()

#G(2) for Galvo
            delay_1 = g2(total_photon_arr)

            bins = np.arange(int(-1.5*interval+25e2), int(1.5*interval-25e2+5e3), 5e3)
            g_2_hist, bin_edges = np.histogram(delay_1, bins=bins)

            bins = (bin_edges[:-1]+bin_edges[1:])/2

# best fit of data
            popt_g2, pcov = curve_fit(g_2_fit, bins, (g_2_hist - np.min(g_2_hist)), 
                                      (np.max(g_2_hist[int((interval / 5e3) - 50):int((interval / 5e3) + 50)]), 
                                       np.max(g_2_hist), 10, 1 / mean_life_x, interval), maxfev=100000)
            p_sigma_g_2_spad = np.sqrt(np.diag(pcov))

            g2_value = (popt_g2[0]) / (popt_g2[1])
            g2_uncertainty = g2_value * np.sqrt((p_sigma_g_2_spad[0] / popt_g2[0])**2 + (p_sigma_g_2_spad[1] / popt_g2[1])**2)

            if plotsshow:
                plt.figure(dpi=300)  # Increase figure size and DPI for higher quality, make it square
                plt.plot(np.array(bins)/1e3, g_2_hist-np.min(g_2_hist), label="G(2) Histogram")
                plt.plot(np.array(bins)/1e3, (g_2_fit(bins, *popt_g2)), 'r--', label="G(2) Fit")

                plt.grid(axis='y', alpha=0.75)
                plt.xlabel('Delay time (ns)')
                plt.ylabel('Intensity (a.u.)')
                plt.yticks()
                plt.ylim(0)
                plt.minorticks_on()
                plt.xlim(-1.5*interval/1e3,1.5*interval/1e3)
                plt.legend()
                plt.show()

            print("BX QY =", g2_value, "with error:", g2_uncertainty)
            g_2.append(g2_value)
            g_2_err.append(g2_uncertainty)
            g_2_err_con = g2_uncertainty

# Calculates and plots the BX spectrum
            total_detect_arr = total_photon_arr_wave[:,1]
            time_total_arr = total_photon_arr_wave[:,0]

            time_gate_1 = 5*mean_life_bx
            time_gate_2 = 5*mean_life_x+time_gate_1

# Here we almost use the same tactic as with the G(2) but with only 1 extra list, one index up, so the time differnce and the differnce in microtime is put into a list from which the detection with BX qaulifications get extracted and plotted
            bx_spectrum_arr = np.vstack(((time_total_arr[1:]-time_total_arr[:-1]), total_detect_arr[:len(total_detect_arr)-1],(np.mod((time_total_arr[:-1]),interval)-np.mod((time_total_arr[1:]),interval)),(np.mod((time_total_arr[:-1]),interval)), (np.mod((time_total_arr[1:]),interval)))).astype(np.int64).T 

            bx_decay_arr = bx_spectrum_arr[(bx_spectrum_arr[:,0] < interval)*(bx_spectrum_arr[:,2] < 0)*(bx_spectrum_arr[:,3] < time_gate_1)*(bx_spectrum_arr[:,4] < time_gate_2)][:,3] 
            x_decay_arr = bx_spectrum_arr[(np.concatenate(([False],(bx_spectrum_arr[1:,0] < interval)))*(bx_spectrum_arr[:,2] > 0))][:,3] 
            x_spectrum_arr = bx_spectrum_arr[(np.concatenate(([False],(bx_spectrum_arr[1:,0] < interval)))*(bx_spectrum_arr[:,2] > 0))][:,1] 
            bx_spectrum_arr = bx_spectrum_arr[(bx_spectrum_arr[:,0] < interval)*(bx_spectrum_arr[:,2] < 0)*(bx_spectrum_arr[:,3] < time_gate_1)*(bx_spectrum_arr[:,4] < time_gate_2)][:,1]
            bx_spectrum_arr, x_spectrum_arr = bx_spectrum_arr[bx_spectrum_arr > 0] ,x_spectrum_arr[x_spectrum_arr > 0] 

# Fitting the BX decay curve
            bins = np.arange(0, int(3*mean_life_bx)+300,30)
            decay_hist, bin_edges = np.histogram(bx_decay_arr, bins=bins)
            
            bins = (bin_edges[:-1]+bin_edges[1:])/2

            bins = bins[decay_hist != 0]
            decay_hist = decay_hist[decay_hist != 0]

            popt, pcov = curve_fit(exp_func, bins, decay_hist, (10,mean_life_bx,10),maxfev = 100000, sigma = np.sqrt(decay_hist))
            p_sigma_decay_bx_galvo = np.sqrt(np.diag(pcov))

            bx_decay_lambda_g.append(2*delta_lambda_apd)
            bx_decay_time_g.append(run_time_in_s)
            bx_decay_fit_g.append((popt[1]) / ((popt_g2[0]) / (popt_g2[1])))
            bx_decay_err_g.append(
                bx_decay_fit_g[-1] * np.sqrt(
                    (p_sigma_decay_bx_galvo[1] / popt[1])**2 +
                    (g_2_err_con / ((popt_g2[0]) / (popt_g2[1])))**2
                )
            )
            if plotsshow:
                plt.figure(dpi=300)  # Increase figure size and DPI for higher quality, make it square
                plt.scatter(np.array(bins)/1e3, decay_hist, label="BX Micro Time Histogram", color="blue", s=10)
                plt.plot(np.array(bins)/1e3, exp_func(bins, *popt), label="Exponential Fit", linestyle='--', color="red")
                plt.yscale("log")
                plt.ylim(0,)
                plt.xlim(0, int(3 * mean_life_bx)/1e3)
                plt.xlabel("Time (ns)")
                plt.ylabel("log(Counts) (a.u.)")
                plt.legend()
                plt.minorticks_on()
                plt.show()

# Fitting the BX spectrum
            bins = np.arange(scanning_min_input, scanning_max_input+2,2)
            bx_spectrum_hist, bin_edges = np.histogram(bx_spectrum_arr, bins=bins)

            bins = (bin_edges[:-1]+bin_edges[1:])/2

# best fit of data
            popt, pcov = curve_fit(gaussian_box, bins, normalize(bx_spectrum_hist), (1.0, mean_bx, standard_deviation_bx,0),maxfev = 100000, bounds=((0,0,0,0),(np.inf,np.inf,np.inf,np.inf)), absolute_sigma= False)
            p_sigma_bx_galvo = np.sqrt(np.diag(pcov))

            bx_spec_lambda_g.append(2*delta_lambda_apd)
            bx_spec_time_g.append(run_time_in_s)                
            bx_spec_fit_g.append((popt[1]))
            bx_spec_err_g.append((p_sigma_bx_galvo[1]))

            if plotsshow:
                plt.figure(dpi=300)  # Increase figure size and DPI for higher quality, make it square
                plt.scatter(bins ,normalize(bx_spectrum_hist),c="b", s=2)
                plt.plot(bins,(gaussian_box(bins,*popt)),c="b")

# Fitting the X spectrum
            bins = np.arange(scanning_min_input, scanning_max_input+2,2)
            x_spectrum_hist, bin_edges = np.histogram(x_spectrum_arr, bins=bins)

            bins = (bin_edges[:-1]+bin_edges[1:])/2

# best fit of data
            popt, pcov = curve_fit(gaussian_box, bins, normalize(x_spectrum_hist), (1.0, mean_x, standard_deviation_x,0),maxfev = 100000, bounds=((0,0,0,0),(np.inf,np.inf,np.inf,np.inf)), absolute_sigma= False)
            p_sigma_x_galvo = np.sqrt(np.diag(pcov))

            if plotsshow:
                plt.scatter(bins,normalize(x_spectrum_hist),c="r", s=2)
                plt.plot(bins,(gaussian_box(bins,*popt)),c="r")

                plt.legend(["BX meausurments","BX fit","X measurments","X fit"], loc= "lower right")
                plt.xlabel("Wavelength (nm)")
                plt.ylabel("Intensity (norm.)")
                plt.ylim(0,)
                plt.xlim(scanning_min_input,scanning_max_input)
                plt.minorticks_on()
                plt.show()
# Plots the decay curve
# Fitting the X decay curve

            x_decay_arr = np.mod(total_photon_arr[:,0][np.concatenate(([False],np.diff((total_photon_arr[:,0]/interval).astype(int)) > 0))],interval)
            bins = np.arange(0, int(4*mean_life_x+1000),1000)
            decay_hist, bin_edges = np.histogram(x_decay_arr, bins=bins)

            bins = (bin_edges[:-1]+bin_edges[1:])/2

            bins = bins[decay_hist != 0]
            decay_hist = decay_hist[decay_hist != 0]

            popt, pcov = curve_fit(exp_func, bins, decay_hist, (10,mean_life_x,100), bounds=((0,0,0),(np.inf,np.inf,np.inf)),maxfev = 1000000, sigma = np.sqrt(decay_hist))
            p_sigma_decay_x_galvo = np.sqrt(np.diag(pcov))

            x_decay_lambda_g.append(2*delta_lambda_apd)
            x_decay_time_g.append(run_time_in_s)
            x_decay_fit_g.append((popt[1]))
            x_decay_err_g.append((p_sigma_decay_x_galvo[1]))

            if plotsshow:
                plt.figure(dpi=300)  # Increase figure size and DPI for higher quality, make it square
                plt.scatter(np.array(bins)/1e3, decay_hist, label="X Micro Time Histogram", color="blue", s=10)
                plt.plot(np.array(bins)/1e3, exp_func(bins, *popt), label="Exponential Fit", linestyle='--', color="red")
                plt.yscale("log")
                plt.ylim(0,)
                plt.xlim(0, int(4 * mean_life_x/1e3))
                plt.xlabel("Time (ns)")
                plt.ylabel("log(Counts) (a.u.)")
                plt.legend()
                plt.minorticks_on()
                plt.show()
            
            darkscounted.append(darkrate*run_time_in_s*2)
            bx_count_list.append(len(bx_decay_arr))
            x_count_list.append(len(total_detect_arr))
# -------------------------

In [ ]:
#Simulation of the setup with the SPAD_Lambda
# IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import random 
from scipy.stats import poisson
from scipy.optimize import curve_fit

QD_constants = True
Custome_constants = False

if QD_constants:
# CONSTANTS
# Probability of absorbing photons based on Poisson statistics
    exp_abs = 0.1
    zero_chance = poisson.pmf(0, exp_abs)  # Probability of absorbing 0 photons
    one_chance = poisson.pmf(1, exp_abs)  # Probability of absorbing 1 photon

# Laser and detection system constants
    freq_laser = 2.5  # Frequency of the laser in MHz
    interval = (1/freq_laser)*1e6 #Calculates the interval between pulses in ps
    mean_x = 633  # Mean wavelength for X state (nm)
    standard_deviation_x = 10  # Standard deviation for X state (nm)
    mean_bx = 629  # Mean wavelength for BX state (nm)
    standard_deviation_bx = 10  # Standard deviation for BX state (nm)

# State lifetimes in picoseconds (ps)
    mean_life_x = int(24 * 1e3)  # Mean lifetime of X state
    rad = 0.105  # Ratio of BX states decaying radiatively
    mean_life_rd_bx = int((1/(2*rad))*1e3) # Mean lifetime of BX state
    mean_life_bx = 1 / ((1 / mean_life_rd_bx)/rad)
    mean_life_a_bx = 1 / ((1 / mean_life_bx) * ((1 - rad) / rad))  # Non-radiative BX lifetime

# Wavelength detection range
    scanning_min = 590  # Minimum wavelength (nm)
    scanning_max = 670  # Maximum wavelength (nm)

# The amount of sensors in the SPAD
    amount_of_sensors = int((scanning_max-scanning_min)/2)
    delta_lambda = ((scanning_max-scanning_min)/amount_of_sensors) #The amount of wavelength that falls on the APD

# System parameters
    g2_times = 4  # Number of detection events considered in g(2) calculation
    down_time = 25e3  # Dead time of a pixel (ps)
    cross_talk_threshold = 1000 #Time after a laser pulse in which all photons get thrown away (ps)

# Simulation configurations
    run_time_list = np.arange(900,0,-60) # Simulation runtimes (s)
    multiplier_list = [1] # 1 for 5 V, 2 for 7 V, 3 for 9 V
    system_eff_list = [0.1] # List of system effs corresponding to the voltage modes
    ratio_list = [0.105]*15
    repeats = 10 # Number of simulation repetitions

# Darkcounts groups
# Percentages of pixels in the group
    group1 = 0.5
    group2 = 0.25
    group3 = 0.2
    group4 = 0.05

# Amount of dark counts per second per group per pixel
    darkcounts_group1 = 31
    darkcounts_group2 = 34
    darkcounts_group3 = 35
    darkcounts_group4 = 40

if Custome_constants:
# CONSTANTS
# Probability of absorbing photons based on Poisson statistics
    exp_abs = 0.1
    zero_chance = poisson.pmf(0, exp_abs)  # Probability of absorbing 0 photons
    one_chance = poisson.pmf(1, exp_abs)  # Probability of absorbing 1 photon

# Laser and detection system constants
    freq_laser = 1  # Frequency of the laser in MHz
    interval = (1/freq_laser)*1e6 #Calculates the interval between pulses in ps
    mean_x = 623  # Mean wavelength for X state (nm)
    standard_deviation_x = 10  # Standard deviation for X state (nm)
    mean_bx = 620  # Mean wavelength for BX state (nm)
    standard_deviation_bx = 10  # Standard deviation for BX state (nm)

# State lifetimes in picoseconds (ps)
    mean_life_x = int(20 * 1e3)  # Mean lifetime of X state
    mean_life_bx = int(2 * 1e3)  # Mean lifetime of BX state
    rad = 0.1  # Ratio of BX states decaying radiatively
    mean_life_a_bx = 1 / ((1 / mean_life_bx) * ((1 - rad) / rad))  # Non-radiative BX lifetime

# Wavelength detection range
    scanning_min = 580  # Minimum wavelength (nm)
    scanning_max = 660  # Maximum wavelength (nm)

# The amount of sensors in the SPAD
    amount_of_sensors = int((scanning_max-scanning_min)/2)
    delta_lambda = ((scanning_max-scanning_min)/amount_of_sensors) #The amount of wavelength that falls on the APD

# System parameters
    g2_times = 4  # Number of detection events considered in g(2) calculation
    down_time = 25e3  # Dead time of a pixel (ps)
    cross_talk_threshold = 1000 #Time after a laser pulse in which all photons get thrown away (ps)

# Simulation configurations
    run_time_list = [300]  # Simulation runtimes (s)
    multiplier_list = [1,2,3] # 1 for 5 V, 2 for 7 V, 3 for 9 V
    system_eff_list = [0.1,0.1075,0.115] # List of system effs corresponding to the voltage modes
    repeats = 100 # Number of simulation repetitions

# Darkcounts groups
# Percentages of pixels in the group
    group1 = 0.5
    group2 = 0.25
    group3 = 0.2
    group4 = 0.05

# Amount of dark counts per second per group per pixel
    darkcounts_group1 = 31
    darkcounts_group2 = 34
    darkcounts_group3 = 35
    darkcounts_group4 = 40

non_uni_darkcounts = True
afterpulsingfilter = True
plotsshow = False

# DEFINITIONS

def decay(mean_lifetime):
    """
    Generate a random lifetime using an exponential distribution.
    :param mean_lifetime: Average lifetime of the state (ps)
    :return: Random lifetime (ps)
    """
    return int(np.random.exponential(mean_lifetime))

def wavelength(mean, sd):
    """
    Generate a random wavelength using a Gaussian distribution.
    :param mean: Mean wavelength (nm)
    :param sd: Standard deviation of the wavelength (nm)
    :return: Random wavelength (nm)
    """
    return np.random.normal(mean, sd)

def mirrorpos(t):
    """
    Calculate the mirror position and corresponding center wavelength at time t.
    :param t: Time (s)
    :return: Center wavelength at time t (nm)
    """
    h = t % freq_mirror
    return scanning_min_input + abs(scanning_max_input - scanning_min_input - 2 * (h * ((scanning_max_input - scanning_min_input) / freq_mirror)))

def g2(time_total):
    """
    Compute g(2) by calculating time differences between detections.
    :param time_total: Array of detections as [time, detector]
    :return: Time differences for g(2) calculation
    """
    delay1 = []
    for i in range(g2_times):
        if len(delay1) == 0 and i != 0:
            delay1 = time_total[i:] - time_total[:-i]
            delay1 = delay1[(abs(delay1[:, 0]) <= 1.5 * interval) * (delay1[:, 1] != 0)]
            delay1 = np.concatenate((delay1, time_total[:-i] - time_total[i:]))
        elif i != 0:
            delay1 = np.concatenate((delay1, time_total[i:] - time_total[:-i]))
            delay1 = delay1[(abs(delay1[:, 0]) <= 1.5 * interval) * (delay1[:, 1] != 0)]
    return delay1[:, 0]

def double_exp_func(x, a, b, d, e, c):
    """
    Double exponential function for fitting decay curves.
    :param x: Time (ps)
    :param a, b, d, e, c: Fit parameters
    :return: Function value
    """
    return a * np.exp(-(1/b) * x) + d * np.exp(-(1/e) * x) + c

def exp_func(x, a, b, c):
    """
    Single exponential function for fitting decay curves.
    :param x: Time (ps)
    :param a, b, c: Fit parameters
    :return: Function value
    """
    return a * np.exp(-(1/b) * x) + c

def normalize(arr):
    """
    Normalize an array to range [0, 1].
    :param arr: Input array
    :return: Normalized array
    """
    return (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

def gaussian(x, c, mu, sigma, d):
    """
    Gaussian function for fitting spectral data.
    :param x: Wavelength (nm)
    :param c, mu, sigma, d: Fit parameters
    :return: Function value
    """
    return abs(c * np.exp(-(x - mu)**2 / (2 * sigma**2))) + d

def gaussian_plusX(x, c, mu, sigma, d, a, mux, sigmax):
    """
    Combined Gaussian function for fitting BX and X spectra.
    :param x: Wavelength (nm)
    :param c, mu, sigma, d, a, mux, sigmax: Fit parameters
    :return: Function value
    """
    return (c * np.exp(-(x - mu)**2 / (2 * sigma**2))) + \
           (a * np.exp(-(x - mux)**2 / (2 * sigmax**2))) + d

def decay_curve_per_detect(decay_list):
    """
    Calculate the dark count rate for each pixel based on decay curves.
    :param decay_list: List of detections [time, detector]
    :return: Dark count rates for each pixel
    """
    darkrate_detect = []
    for i in range(amount_of_sensors + 1):
        detect_list = decay_list[decay_list[:, 1] == i ]
        detect_list = np.array(detect_list).astype(np.int64)
        detect_list = np.mod(detect_list[:, 0], interval)
        bins = np.linspace(0, interval+1000, 1000)
        decay_hist, _ = np.histogram(detect_list, bins=bins)
        darkrate_detect.append(((np.sum(decay_hist[-100:]) / 100) * len(decay_hist)))
    return darkrate_detect[:-1]

def gaussian_box(x, c, mu, sigma):
    """
    Gaussian convoluted with a box function for fitting.
    :param x: Wavelength (nm)
    :param c, mu, sigma: Fit parameters
    :return: Function value
    """
    gaussian = c * np.exp(-(x - mu)**2 / (2 * sigma**2))
    box = np.ones(int(delta_lambda_apd))
    return np.convolve(box, gaussian, 'same')

def g_2_fit(x, a, b, d, e, f):
    """
    Model function for fitting g(2) curves.
    :param x: Time difference (ps)
    :param a, b, d, e, f: Fit parameters
    :return: Function value
    """
    return a * np.exp(-1 * abs(x) * e) + \
           b * np.exp(-1 * abs(x - f) * e) + \
           b * np.exp(-1 * abs(x + f) * e) + d\

# ----------LOOP SPAD----------

# Lists where the output is put in
x_spec_fit, x_spec_err, bx_spec_fit, bx_spec_err, bx_decay_fit, bx_decay_err, x_decay_fit, x_decay_err = [], [], [], [], [], [], [], []
x_spec_time, x_spec_volt, bx_spec_time, bx_spec_volt, bx_decay_time, bx_decay_volt, x_decay_time, x_decay_volt = [], [], [], [], [], [], [], []
darkscounted, bx_count_list, x_count_list = [], [], []
g_2, g_2_err = [], []

for j in range(len(run_time_list)):
# --------Change in Runtime--------
    run_time_in_s = run_time_list[j]
    run_time_in_s = int(run_time_in_s)
    run_time = int(run_time_in_s*1e12) #run_time of the entire experiment in ps
    pulses = int(run_time/interval) #Calculates the total amount of pulses in the entire experiment in ps

    rad = ratio_list[j]  # Ratio of BX states decaying radiatively

    mean_life_bx = 1 / ((1 / mean_life_rd_bx)/rad)
    mean_life_a_bx = 1 / ((1 / mean_life_bx) * ((1 - rad) / rad))  # Non-radiative BX lifetime

# --------Change in Voltage--------
    for k in range(len(multiplier_list)):
        multiplier = multiplier_list[k]
        system_eff = system_eff_list[k]
        

        for l in range(repeats):
            print(j+1,"/",len(run_time_list),",",k+1,"/",len(multiplier_list),",",l+1,"/",repeats)
            darkcounts_list = np.arange(0,amount_of_sensors)
            random.shuffle(darkcounts_list)
            darkcounts_detectlist_1 = darkcounts_list[:int(amount_of_sensors*group1)]
            darkcounts_detectlist_2 = darkcounts_list[int(amount_of_sensors*group1):int(amount_of_sensors*(group1+group2))]
            darkcounts_detectlist_3 = darkcounts_list[int(amount_of_sensors*(group1+group2)):int(amount_of_sensors*(group1+group2+group3))]
            darkcounts_detect_list_4 = darkcounts_list[int(amount_of_sensors*(group1+group2+group3)):int(amount_of_sensors)]
            
            darks_1_nonuni3 = (np.random.choice(darkcounts_detectlist_2, size=int(len(darkcounts_detectlist_2)*darkcounts_group2*run_time_in_s*multiplier)))

            darks_1_nonuni4 = (np.random.choice(darkcounts_detectlist_1, size=int(len(darkcounts_detectlist_1)*darkcounts_group1*run_time_in_s*multiplier)))

            darks_1_nonuni1 = (np.random.choice(darkcounts_detect_list_4,  size=int(len(darkcounts_detect_list_4)*darkcounts_group4*run_time_in_s*multiplier)))

            darks_1_nonuni2 = (np.random.choice(darkcounts_detectlist_3, size=int(len(darkcounts_detectlist_3)*darkcounts_group3*run_time_in_s*multiplier)))

            darks_1_nonuni = np.concatenate((darks_1_nonuni1,darks_1_nonuni2,darks_1_nonuni3,darks_1_nonuni4))

            number_x_spad = int(np.random.normal(((pulses)*(1-zero_chance))))
            number_bx_spad = int(np.random.normal(((pulses)*(1-zero_chance-one_chance))))
            number_a_bx_spad = int(number_bx_spad*(1-rad))

            print(number_x_spad,number_bx_spad-number_a_bx_spad,number_a_bx_spad)    
# Determines the amount of X and BX detections and defines the decayrates

# Generates a list of the wavelengths for the X and BX
            x_wavelengths_spad = np.random.normal(mean_x, standard_deviation_x, number_x_spad)
            bx_wavelengths_spad = np.random.normal(mean_bx, standard_deviation_bx, number_bx_spad-number_a_bx_spad)
            bx_a_wavelengths_spad = [-1]*int(number_a_bx_spad*system_eff)

# Turns the wavelengths into specific detectors
            detect_x_spad = (((x_wavelengths_spad - (scanning_min)) / delta_lambda)).astype(np.int32)
            detect_bx_spad = (((bx_wavelengths_spad - (scanning_min)) / delta_lambda)).astype(np.int32) 
            detect_x_1_spad = detect_x_spad[:int((number_x_spad-number_bx_spad)*system_eff)]
            detect_x_2_spad = detect_x_spad[number_x_spad-number_bx_spad:number_x_spad - number_a_bx_spad]
            detect_x_3_spad = detect_x_spad[number_x_spad - int(number_a_bx_spad*system_eff):]

# Creates a lists with lifetimes for the X where no BX is detected, and a list of zeros for the BX. It also generates a little less of these becayse of system eff. this will later be forced on the X with a BX as well
            photon_bx_arr_1_spad = (np.zeros(int((number_x_spad-number_bx_spad)*system_eff))).astype(int)
            photon_x_arr_1_spad= (-mean_life_x*np.log(np.random.uniform(0,1,(int((number_x_spad-number_bx_spad)*system_eff)))))

# Creates 2 lists with lifetimes for the X and BX which decay radiative
            photon_bx_arr_2_spad = (-mean_life_bx*np.log(np.random.uniform(0,1,number_bx_spad-number_a_bx_spad)))
            print(len(photon_bx_arr_2_spad))
            photon_x_arr_2_spad =(-mean_life_x*np.log(np.random.uniform(0,1,number_bx_spad-number_a_bx_spad)))

# Creates 2 lists with lifetimes for the X and BX which decay via auger
            photon_bx_arr_3_spad = (-mean_life_a_bx*np.log(np.random.uniform(0,1,int(number_a_bx_spad*system_eff))))
            photon_x_arr_3_spad =(-mean_life_x*np.log(np.random.uniform(0,1,int(number_a_bx_spad*system_eff))))

# All lists get added togheter so we get one array with all the data. (BX_lifetime,X_lifetime,BX_detector,X_detector)
            photon_arr_1_spad = np.vstack((photon_bx_arr_1_spad,photon_x_arr_1_spad,photon_bx_arr_1_spad,detect_x_1_spad)).T
            photon_arr_2_spad = np.vstack((photon_bx_arr_2_spad,photon_x_arr_2_spad,detect_bx_spad,detect_x_2_spad)).T
            photon_arr_3_spad = np.vstack((photon_bx_arr_3_spad,photon_x_arr_3_spad,bx_a_wavelengths_spad,detect_x_3_spad)).T
            photon_arr_spad = np.vstack((photon_arr_1_spad,photon_arr_2_spad,photon_arr_3_spad))
# creates a list of pulse times and choices random times out of a list to choose the arrival times of the photons

            rng = np.random.default_rng()
            numbertimes = 600
            time_total_parts = []

            photons_per_part = int((len(photon_arr_spad) + numbertimes) / numbertimes)

            for j in range(numbertimes):
                start = int(j * pulses / numbertimes+pulses)
                stop = int((j + 1) * pulses / numbertimes+pulses)
                time_total_part = (rng.choice(np.arange(start, stop), size=photons_per_part, replace=False) * interval)
                time_total_parts.append(time_total_part)

            time_total = np.concatenate(time_total_parts)[:len(photon_arr_spad)]

# Makes is so that the BX lifetime gets added to the X lifetime, where they are both within 1 interval
            photon_arr_spad[:, :2] = np.cumsum(photon_arr_spad[:, :2], axis=1)

# creates a mask, so where there are liftimes on the list a random time is added. But in such a way that the BX is always with a X.
            macro_ = np.vstack((time_total, time_total)).T

            mask = photon_arr_spad[:,:2] != 0

            photon_arr_spad[:,:2][mask] += macro_[mask]

# System_eff for X with BX

# Here some of the enetries for the X with a BX and BX lists gets destroyed due to system eff.
            remain_numb = int((number_bx_spad-number_a_bx_spad)*system_eff)
            print(remain_numb)
            delete_numb = int(number_bx_spad-number_a_bx_spad-remain_numb)
            system_eff_arr = [1]*remain_numb + [0]*delete_numb
            system_eff_arr = np.array(system_eff_arr).astype(int)
            np.random.shuffle(system_eff_arr)
            photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),0] = photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),0] * system_eff_arr
            photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),2] = photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),2] * system_eff_arr
            np.random.shuffle(system_eff_arr)
            photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),1] = photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),1] * system_eff_arr
            photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),3] = photon_arr_spad[int((number_x_spad - number_bx_spad)*system_eff):len(photon_arr_spad)-len(photon_bx_arr_3_spad),3] * system_eff_arr

# Darkcounts
            if non_uni_darkcounts:  

# This part makes the non_uniform_darkcounts possible, it creates list with random detectors according to a simplification of the typical distribution of dark count rate of the SPAD
                start = 0
                stop = run_time

                darks_3 = np.random.uniform(start,stop,int(darkcounts_group2*len(darkcounts_detectlist_2)*run_time_in_s*multiplier))

                darks_4 = np.random.uniform(start,stop,int(darkcounts_group1*len(darkcounts_detectlist_1)*run_time_in_s*multiplier))

                darks_1 = np.random.uniform(start,stop,int(darkcounts_group4*len(darkcounts_detect_list_4)*run_time_in_s*multiplier))

                darks_2 = np.random.uniform(start,stop,int(darkcounts_group3*len(darkcounts_detectlist_3)*run_time_in_s*multiplier))

                darks_2 = np.concatenate((darks_1,darks_2,darks_3,darks_4))

                total_darkcount_arr = np.vstack((darks_2, darks_1_nonuni)).T

# Here everything is added toghether zeros are filtered out and it gets sorted
                totol_bx_photon_arr = np.vstack((photon_arr_spad[:,0],photon_arr_spad[:,2])).T
                totol_x_photon_arr = np.vstack((photon_arr_spad[:,1],photon_arr_spad[:,3])).T
                total_photon_arr = np.vstack((totol_bx_photon_arr, totol_x_photon_arr,total_darkcount_arr))

                total_photon_arr = total_photon_arr[np.where(total_photon_arr[:,0] != 0)]
                total_photon_arr = total_photon_arr[np.argsort(total_photon_arr[:, 0])]
                total_photon_arr_spad = total_photon_arr

            else:
# Here everything is added toghether zeros are filtered out and it gets sorted
                totol_bx_photon_arr = np.vstack((photon_arr_spad[:,0],photon_arr_spad[:,2])).T
                totol_x_photon_arr = np.vstack((photon_arr_spad[:,1],photon_arr_spad[:,3])).T
                total_photon_arr = np.vstack((totol_bx_photon_arr, totol_x_photon_arr))

                total_photon_arr = total_photon_arr[np.where(total_photon_arr[:,0] != 0)]
                total_photon_arr = total_photon_arr[np.argsort(total_photon_arr[:, 0])]
                total_photon_arr_spad = np.vstack((total_photon_arr_spad,total_photon_arr))  

# Remove fake wavelength bx auger
            total_photon_arr_spad = total_photon_arr_spad[total_photon_arr_spad[:,1] > -1]

# Applies the crosstalk threshold

            total_photon_arr_spad = total_photon_arr_spad[np.concatenate(([False],((np.diff(total_photon_arr_spad[:,0])) > cross_talk_threshold)))] 

# Applies deadtime filter

            total_photon_arr_spad = total_photon_arr_spad[~((np.concatenate(([False],((np.diff(total_photon_arr_spad[:,0])) < down_time))))*(np.concatenate(([True],((np.diff(total_photon_arr_spad[:,1])) == 0)))))] 

            if afterpulsingfilter:
                total_photon_arr_spad = total_photon_arr_spad[~(np.concatenate(([False],((np.diff(total_photon_arr_spad[:,0]) < interval)*(np.diff(total_photon_arr_spad[:,1]) == 0)*(np.diff(np.mod(total_photon_arr_spad[:,0],interval))>0)))))]

# --------End of Simulation--------

# Plots the decay curve
# Fitting the X decay curve

            x_decay_arr = np.mod(total_photon_arr_spad[:,0][np.concatenate(([False],np.diff((total_photon_arr_spad[:,0]/interval).astype(int)) > 0))],interval)
            bins = np.arange(0, int(4*mean_life_x+1000),1000)
            decay_hist, bin_edges = np.histogram(x_decay_arr, bins=bins)

            bins = (bin_edges[:-1]+bin_edges[1:])/2

            bins = bins[decay_hist != 0]
            decay_hist = decay_hist[decay_hist != 0]

            popt, pcov = curve_fit(exp_func, bins, decay_hist, (10,mean_life_x,100), bounds=((0,0,0),(np.inf,np.inf,np.inf)),maxfev = 1000000, sigma = np.sqrt(decay_hist))
            p_sigma_decay_x_galvo = np.sqrt(np.diag(pcov))

            x_decay_volt.append(multiplier)
            x_decay_time.append(run_time_in_s)
            x_decay_fit.append((popt[1]))
            x_decay_err.append((p_sigma_decay_x_galvo[1]))

            if plotsshow:
                plt.plot(bins,decay_hist)
                plt.plot(bins, exp_func(bins,*popt))
                plt.yscale("log")
                plt.xlim(0,int(4*mean_life_x))
                plt.ylim(0,)
                plt.show()

# Calculates and plots the X spectrum

            def gaussian_darkcounts(x, c, mu, sigma, n):
                background =(n)*np.array(decay_hist_per_pixel)/10
                res =  (c * np.exp( - (x - mu)**2.0 / (2.0 *  sigma**2.0) )) + background
                return res

            decay_hist_per_pixel = decay_curve_per_detect(total_photon_arr_spad)

            total_detect_arr = total_photon_arr_spad[:,1]
            bins = np.arange(0,amount_of_sensors+1)
            emission_hist, bin_edges = np.histogram(total_detect_arr, bins=bins)

            bins = np.arange(0,amount_of_sensors)
            bins1 = np.arange(scanning_min+delta_lambda/2,scanning_max+delta_lambda/2,delta_lambda)

# best fit of data
            poptx, pcov = curve_fit(gaussian_darkcounts, bins, normalize(emission_hist), (1000, (mean_x-scanning_min-delta_lambda/2)/delta_lambda, standard_deviation_x/delta_lambda,0),bounds=((0,0,0,0),(np.inf,np.inf,np.inf,np.inf)), maxfev = 100000)
            p_sigma_x_spad = np.sqrt(np.diag(pcov))

            x_spec_time.append(run_time_in_s)
            x_spec_volt.append(multiplier)
            x_spec_fit.append(poptx[1]*2+scanning_min+delta_lambda/2)
            x_spec_err.append(p_sigma_x_spad[1]*2)

            darkscounted.append(np.sum(decay_hist_per_pixel))
            

            if plotsshow:            
                plt.figure(dpi=300)  # Increase figure size and DPI for higher quality, make it square
                plt.plot(bins1, gaussian_darkcounts(bins, *poptx), c="darkorange", linewidth=2, label="Spectrum Fit")
                plt.scatter(bins1, normalize(emission_hist), c="darkblue", s=30, label="Spectrum Measurements", zorder=5)
                plt.legend(loc="lower right")
                plt.xlabel("Wavelength (nm)")
                plt.ylabel("Intensity (norm.)")
                plt.ylim(0,)
                plt.xlim(scanning_min, scanning_max)
                plt.minorticks_on()
                plt.xticks()
                plt.yticks()
                plt.show()

#G(2) for SPAD
            delay_1 = g2(total_photon_arr_spad)

            bins = np.arange(int(-1.5*interval+25e2), int(1.5*interval-25e2+5e3), 5e3)
            g_2_hist, bin_edges = np.histogram(delay_1, bins=bins)
            
            bins = np.arange(int(-1.5*interval+25e2), int(1.5*interval-25e2), 5e3)

# best fit of data
            mask = np.ones_like(bins, dtype=bool)
            mask[mask.shape[0] // 2] = 0
            popt_g2, pcov = curve_fit(g_2_fit, bins[mask], (g_2_hist - np.min(g_2_hist))[mask], 
                                      (np.max(g_2_hist[mask][int((interval / 5e3) - 50):int((interval / 5e3) + 50)]), 
                                       np.max(g_2_hist[mask]), 10, 1 / mean_life_x, interval), maxfev=100000)
            p_sigma_g_2_spad = np.sqrt(np.diag(pcov))

            g2_value = (popt_g2[0]) / (popt_g2[1])
            g2_uncertainty = g2_value * np.sqrt((p_sigma_g_2_spad[0] / popt_g2[0])**2 + (p_sigma_g_2_spad[1] / popt_g2[1])**2)

            if plotsshow:
                plt.plot(bins[mask], (g_2_fit(bins[mask], *popt_g2)), 'r--')
                plt.plot(bins[mask], (g_2_hist - np.min(g_2_hist))[mask])

                plt.grid(axis='y', alpha=0.75)
                plt.xlabel('Delay time [ps]')
                plt.ylabel('Counts [a.u.]')
                plt.title('g(2)')
                plt.ylim(0)
                plt.xlim(-1.5 * interval, 1.5 * interval)
                plt.show()

            print("BX QY =", g2_value, "with error:", g2_uncertainty)
            g_2.append(g2_value)
            g_2_err.append(g2_uncertainty)
            g_2_err_con = g2_uncertainty

            time_total = total_photon_arr_spad[:,0]

            time_gate_1 = 5*mean_life_bx
            time_gate_2 = 5*mean_life_x+time_gate_1

# Here we almost use the same tactic as with the G(2) but with only 1 extra list, one index up, so the time differnce and the differnce in microtime is put into a list from which the detection with BX qaulifications get extracted and plotted
            bx_spectrum_arr = np.vstack((((time_total[1:]-time_total[:-1]), total_detect_arr[:len(total_detect_arr)-1],(np.mod((time_total[:-1]),interval)-np.mod((time_total[1:]),interval), (np.mod((time_total[:-1]),interval)), (np.mod((time_total[1:]),interval)))))).T 
            bx_decay_arr_total = np.vstack((bx_spectrum_arr[(bx_spectrum_arr[:,0] < interval)*(bx_spectrum_arr[:,2] < 0)][:,3],bx_spectrum_arr[(bx_spectrum_arr[:,0] < interval)*(bx_spectrum_arr[:,2] < 0)][:,1])).astype(np.int64).T
            bx_decay_arr = bx_spectrum_arr[(bx_spectrum_arr[:,0] < interval)*(bx_spectrum_arr[:,2] < 0)*(bx_spectrum_arr[:,3] < time_gate_1)*(bx_spectrum_arr[:,4] < time_gate_2)][:,3] 
            x_spectrum_arr = bx_spectrum_arr[np.concatenate(([False],(bx_spectrum_arr[:,0] < interval)*(bx_spectrum_arr[:,2] < 0)*(bx_spectrum_arr[:,3] < time_gate_1)*(bx_spectrum_arr[:,4] < time_gate_2)))[:-1]][:,1] 
            bx_spectrum_arr = bx_spectrum_arr[(bx_spectrum_arr[:,0] < interval)*(bx_spectrum_arr[:,2] < 0)*(bx_spectrum_arr[:,3] < time_gate_1)*(bx_spectrum_arr[:,4] < time_gate_2)][:,1] 

            decay_hist_per_pixel_bx = np.array(decay_curve_per_detect(bx_decay_arr_total))
            # Fitting the BX decay curve
            bins = np.arange(0, int(3*mean_life_bx+300),100)
            decay_hist_BX, bin_edges = np.histogram(bx_decay_arr, bins=bins)

            bins = (bin_edges[:-1]+bin_edges[1:])/2

            bins = bins[decay_hist_BX != 0]
            decay_hist_BX = decay_hist_BX[decay_hist_BX != 0]
            popt, pcov = curve_fit(exp_func, bins, decay_hist_BX, (10, mean_life_bx, 0), sigma=np.sqrt(decay_hist_BX), maxfev=100000)
            p_sigma_bx_spad = np.sqrt(np.diag(pcov))

            bx_decay_time.append(run_time_in_s)
            bx_decay_volt.append(multiplier)
            bx_decay_fit.append((popt[1]) / ((popt_g2[0]) / (popt_g2[1])))
            bx_decay_err.append(
                bx_decay_fit[-1] * np.sqrt(
                    (p_sigma_bx_spad[1] / popt[1])**2 +
                    (g_2_err_con / ((popt_g2[0]) / (popt_g2[1])))**2
                )
            )
           
            if plotsshow:
                plt.figure(dpi=300)  # Increase figure size and DPI for higher quality, make it square
                plt.scatter(np.array(bins)/1e3, decay_hist_BX, label="BX Micro Time Histogram", color="blue", s=10)
                plt.plot(np.array(bins)/1e3, exp_func(bins, *popt), label="Exponential Fit", linestyle='--', color="red")
                plt.yscale("log")
                plt.ylim(0,)
                plt.xlim(0, int(3 * mean_life_bx)/1e3)
                plt.xlabel("Time (ns)")
                plt.ylabel("log(Counts) (a.u.)")
                plt.legend()
                plt.minorticks_on()
                plt.show()

#Fitting the X spectrum
            bins = np.arange(0,amount_of_sensors+1)
            bx_spectrum_hist, bin_edges = np.histogram(bx_spectrum_arr, bins=bins, density=False)
            x_spectrum_hist, bin_edges = np.histogram(x_spectrum_arr, bins=bins, density=False)
            
            bins = np.arange(0,amount_of_sensors)

            bins2 = np.arange(scanning_min+delta_lambda/2,scanning_max+delta_lambda/2,delta_lambda)

            def gaussian_darkcounts(x, c, mu, sigma, d):
                res =  (c * np.exp( - (x - mu)**2.0 / (2.0 *  sigma**2.0) + d )) 
                return res

            popty, pcov = curve_fit(gaussian_darkcounts, bins, x_spectrum_hist, (1, poptx[1], poptx[2],1),maxfev = 10000000, bounds=(((0),(poptx[1]-2),(poptx[2]-2),(0)),(np.inf,poptx[1]+1, poptx[2]+2,np.inf)))
            if plotsshow:
                plt.figure(dpi = 300)
                plt.scatter(bins2,x_spectrum_hist, s=2,c="r")
                plt.plot(bins2,gaussian_darkcounts(bins,*popty),c="r")

#Fitting the BX spectrum
            decay_hist_per_pixel_bx = decay_hist_per_pixel_bx[bx_spectrum_hist != 0]

            bins1 = np.arange(scanning_min+delta_lambda/2,scanning_max+delta_lambda/2,delta_lambda)

            popt, pcov = curve_fit(gaussian_darkcounts, bins, bx_spectrum_hist, (1, (mean_bx-scanning_min)/delta_lambda, standard_deviation_bx/delta_lambda, 0),maxfev = 10000000, bounds=((0,0,0,(0)),(np.inf,np.inf,np.inf,np.inf)))
            p_sigma_bx_spad1 = np.sqrt(np.diag(pcov))

            bx_spec_fit.append(popt[1]*2+scanning_min+delta_lambda/2)
            bx_spec_err.append(p_sigma_bx_spad1[1]*2)
            bx_spec_time.append(run_time_in_s)
            bx_spec_volt.append(multiplier)

            if plotsshow:
                plt.scatter(bins,normalize(x_spectrum_hist),c="r", s=2)
                plt.plot(bins,(gaussian_darkcounts(bins,*popt)),c="r")

                plt.legend(["BX meausurments","BX fit","X measurments","X fit"], loc= "lower right")
                plt.xlabel("Wavelength (nm)")
                plt.ylabel("Intensity (norm.)")
                plt.ylim(0,)
                plt.xlim(scanning_min,scanning_max)
                plt.minorticks_on()
                plt.show()

            bx_count_list.append(len(bx_decay_arr))
            x_count_list.append(len(total_detect_arr))

print(bx_count_list,x_count_list,darkscounted)
# -------------------------

In [ ]:
#Results Galvo setup Run Times

#Results SPAD Voltages
bx_spec_fit_mean, bx_spec_err_mean = [], []
x_spec_fit_mean, x_spec_err_mean = [], []
x_decay_fit_mean, x_decay_err_mean = [], []
bx_decay_fit_mean, bx_decay_err_mean = [], []
bx_spec_std, x_spec_std, x_decay_std, bx_decay_std = [], [], [], []

# Number of measurements per run time
measurements_per_run_time = repeats

# Iterate over the run_time_list
for i in range(len(run_time_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time

    x_spec_std.append(np.std(x_spec_fit_g[start_index:end_index]))
    bx_spec_std.append(np.std(bx_spec_fit_g[start_index:end_index]))
    x_decay_std.append(np.mean(x_decay_fit_g[start_index:end_index]))
    bx_decay_std.append(np.mean(bx_decay_fit_g[start_index:end_index]))

print(x_spec_std)
print(bx_spec_std)
print(x_decay_std)
print(bx_decay_std)

# Mean results for run time with mean error

bx_spec_fit_mean, bx_spec_err_mean = [], []

for i in range(len(run_time_list)):
    time = run_time_list[i]
    mask = []
    for j in range(len(bx_spec_time_g)):
        mask.append(bx_spec_time_g[j] == time)   
    bx_spec_fit_mean.append(np.average(bx_spec_fit_g, weights=mask))
    bx_spec_err_mean.append(np.average(bx_spec_err_g, weights=mask))
plt.errorbar(run_time_list,bx_spec_fit_mean,bx_spec_err_mean,fmt='o', capsize=4, color='crimson', label="Data", marker='D')
plt.errorbar((0,np.max(run_time_list)+np.min(run_time_list)),(mean_x,mean_x),ls = ':', color="dodgerblue")
plt.errorbar((0,np.max(run_time_list)+np.min(run_time_list)),(mean_bx,mean_bx),ls = ':', color="deeppink")

x_spec_fit_mean, x_spec_err_mean = [], []

for i in range(len(run_time_list)):
    time = run_time_list[i]
    mask = []
    for j in range(len(x_spec_time_g)):
        mask.append(x_spec_time_g[j] == time)   
    x_spec_fit_mean.append(np.average(x_spec_fit_g, weights=mask))
    x_spec_err_mean.append(np.average(x_spec_err_g, weights=mask))
plt.errorbar(run_time_list,x_spec_fit_mean,x_spec_err_mean,fmt='o', capsize=4, color='royalblue', label="Data", marker='D')
plt.gca().set_aspect((np.max(run_time_list)+np.min(run_time_list))/5)
plt.xlim(0,(np.max(run_time_list)+np.min(run_time_list)))
if mean_x<mean_bx:
    plt.ylim(mean_x - 1,mean_bx + 1)
else:
    plt.ylim(mean_bx - 1,mean_x + 1)

plt.ylabel("Wavelength (nm)")
plt.xlabel("Run time (s)")
plt.minorticks_on()
plt.show()

x_decay_fit_mean, x_decay_err_mean = [], []

for i in range(len(run_time_list)):
    time = run_time_list[i]
    mask = []
    for j in range(len(x_decay_time_g)):
        mask.append(x_decay_time_g[j] == time)   
    x_decay_fit_mean.append(np.average(x_decay_fit_g, weights=mask))
    x_decay_err_mean.append(np.average(x_decay_err_g, weights=mask))
plt.errorbar(run_time_list,np.array(x_decay_fit_mean)/1e3,np.array(x_decay_err_mean)/1e3,fmt='o', capsize=4, color='royalblue', label="Data", marker='D')
plt.errorbar((0,np.max(run_time_list)+np.min(run_time_list)),(mean_life_x/1e3,mean_life_x/1e3),ls = ':', color="dodgerblue")
plt.gca().set_aspect(((np.max(run_time_list)+np.min(run_time_list)))/2)
plt.xlim((0,np.max(run_time_list)+np.min(run_time_list)))
plt.ylim(23,25)
plt.xlabel("Run time (s)")
plt.ylabel("Lifetime (ns)")
plt.minorticks_on()
plt.show()

bx_decay_fit_mean, bx_decay_err_mean = [], []

for i in range(len(run_time_list)):
    time = run_time_list[i]
    mask = []
    for j in range(len(bx_decay_time_g)):
        mask.append(bx_decay_time_g[j] == time)   
    bx_decay_fit_mean.append(np.average(bx_decay_fit_g, weights=mask))
    bx_decay_err_mean.append(np.average(bx_decay_err_g, weights=mask))
plt.errorbar(run_time_list,np.array(bx_decay_fit_mean)/1e3,np.array(bx_decay_err_mean)/1e3,fmt='o', capsize=4, color='crimson', label="Data", marker='D')
plt.errorbar((0,np.max(run_time_list)+np.min(run_time_list)),(mean_life_rd_bx/1e3,mean_life_rd_bx/1e3),ls = ':', color="deeppink")
plt.gca().set_aspect((np.max(run_time_list)+np.min(run_time_list))/2)
plt.xlim(0,np.max(run_time_list)+np.min(run_time_list))
plt.ylim(mean_life_rd_bx/1e3-1,mean_life_rd_bx/1e3+1)
plt.xlabel("Run time (s)")
plt.ylabel("Lifetime (ns)")
plt.minorticks_on()
plt.show()

# Calculate the average bx_count_list per run time
bx_count_avg = []
x_count_avg = []
darkscounted_avg = []

# Number of measurements per run time
measurements_per_run_time = repeats 

# Iterate over the run_time_list
for i in range(len(run_time_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time
    
    avg_bx_count = np.mean(bx_count_list[start_index:end_index])
    bx_count_avg.append(avg_bx_count)
    
    avg_x_count = np.mean(x_count_list[start_index:end_index])
    x_count_avg.append(avg_x_count)
    
    avg_darkscounted = np.mean(darkscounted[start_index:end_index])
    darkscounted_avg.append(avg_darkscounted)

# Print the average bx_count_list, x_count_list, and darkscounted per run time
print("Average BX Count per Run Time:", bx_count_avg)
print("Average X Count per Run Time:", x_count_avg)
print("Average Dark Counts per Run Time:", darkscounted_avg)

#Results SPAD heralded
print(run_time_list)
print(bx_decay_err_mean)
print(bx_decay_fit_mean)
print(x_decay_err_mean)
print(x_decay_fit_mean)
print(bx_spec_err_mean)
print(bx_spec_fit_mean)
print(x_spec_err_mean)
print(x_spec_fit_mean)

In [ ]:
#Results Galvo Setup Delta Lambda

#Results SPAD Voltages
bx_spec_fit_mean, bx_spec_err_mean = [], []
x_spec_fit_mean, x_spec_err_mean = [], []
x_decay_fit_mean, x_decay_err_mean = [], []
bx_decay_fit_mean, bx_decay_err_mean = [], []
bx_spec_std, x_spec_std, x_decay_std, bx_decay_std = [], [], [], []

# Number of measurements per run time
measurements_per_run_time = repeats

# Iterate over the run_time_list
for i in range(len(delta_lambda_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time

    x_spec_std.append(np.std(x_spec_fit_g[start_index:end_index]))
    bx_spec_std.append(np.std(bx_spec_fit_g[start_index:end_index]))
    x_decay_std.append(np.mean(x_decay_fit_g[start_index:end_index]))
    bx_decay_std.append(np.mean(bx_decay_fit_g[start_index:end_index]))

print(x_spec_std)
print(bx_spec_std)
print(x_decay_std)
print(bx_decay_std)

bx_spec_fit_mean, bx_spec_err_mean = [], []

for i in range(len(delta_lambda_list)):
    lambda_ = delta_lambda_list[i]
    print(lambda_)
    mask = []
    for j in range(len(bx_spec_lambda_g)):
        mask.append(bx_spec_lambda_g[j] == lambda_)  
    bx_spec_fit_mean.append(np.average(bx_spec_fit_g, weights=mask))
    bx_spec_err_mean.append(np.average(bx_spec_err_g, weights=mask))


x_spec_fit_mean, x_spec_err_mean = [], []

for i in range(len(delta_lambda_list)):
    lambda_ = delta_lambda_list[i]
    mask = []
    for j in range(len(x_spec_lambda_g)):
        mask.append(x_spec_lambda_g[j] == lambda_)   
    x_spec_fit_mean.append(np.average(x_spec_fit_g, weights=mask))
    x_spec_err_mean.append(np.average(x_spec_err_g, weights=mask))
plt.errorbar((0,np.max(delta_lambda_list)+np.min(delta_lambda_list)),(mean_x,mean_x),ls = ':', color="dodgerblue")
plt.errorbar((0,np.max(delta_lambda_list)+np.min(delta_lambda_list)),(mean_bx,mean_bx),ls = ':', color="deeppink")
plt.errorbar(delta_lambda_list,x_spec_fit_mean,x_spec_err_mean,fmt='o', capsize=4, color='royalblue', label="Data", marker='D')
plt.errorbar(delta_lambda_list,bx_spec_fit_mean,bx_spec_err_mean,fmt='o', capsize=4, color='crimson', label="Data", marker='D')
plt.xlim(0,np.max(delta_lambda_list)+np.min(delta_lambda_list))
if mean_x<mean_bx:
    plt.ylim(mean_x - 2,mean_bx + 1)
    plt.gca().set_aspect((((np.max(delta_lambda_list)+np.min(delta_lambda_list)))/(-mean_x +mean_bx + 3)))
else:
    plt.ylim(mean_bx - 2,mean_x + 1)
    plt.gca().set_aspect((((np.max(delta_lambda_list)+np.min(delta_lambda_list)))/(-mean_bx +mean_x + 3)))
plt.ylabel("Wavelength (nm)")
plt.xlabel("Delta Lambda (nm)")
plt.minorticks_on()
plt.show()

x_decay_fit_mean, x_decay_err_mean = [], []

for i in range(len(delta_lambda_list)):
    lambda_ = delta_lambda_list[i]
    mask = []
    for j in range(len(x_decay_lambda_g)):
        mask.append(x_decay_lambda_g[j] == lambda_)   
    x_decay_fit_mean.append(np.average(x_decay_fit_g, weights=mask))
    x_decay_err_mean.append(np.average(x_decay_err_g, weights=mask))
plt.errorbar(delta_lambda_list,np.array(x_decay_fit_mean)/1e3,np.array(x_decay_err_mean)/1e3,fmt='o', capsize=4, color='royalblue', label="Data", marker='D')
plt.errorbar((0,np.max(delta_lambda_list)+np.min(delta_lambda_list)),(mean_life_x/1e3,mean_life_x/1e3),ls = ':', color="dodgerblue")
plt.xlim(0,np.max(delta_lambda_list)+np.min(delta_lambda_list))
plt.gca().set_aspect((((np.max(delta_lambda_list)+np.min(delta_lambda_list)))))
plt.ylim((mean_life_x/1e3)-0.5,(mean_life_x/1e3)+0.5)
plt.ylabel("Lifetime (ns)")
plt.xlabel("Delta Lambda (nm)")
# plt.xticks([5,7,9])
plt.minorticks_on()
plt.show()

bx_decay_fit_mean, bx_decay_err_mean = [], []

for i in range(len(delta_lambda_list)):
    lambda_ = delta_lambda_list[i]
    mask = []
    for j in range(len(bx_decay_lambda_g)):
        mask.append(bx_decay_lambda_g[j] == lambda_)   
    bx_decay_fit_mean.append(np.average(bx_decay_fit_g, weights=mask))
    bx_decay_err_mean.append(np.average(bx_decay_err_g, weights=mask))
plt.errorbar(delta_lambda_list,np.array(bx_decay_fit_mean)/1e3,np.array(bx_decay_err_mean)/1e3,fmt='o', capsize=4, color='crimson', label="Data", marker='D')
plt.errorbar((0,np.max(delta_lambda_list)+np.min(delta_lambda_list)),(mean_life_bx/1e3,mean_life_bx/1e3),ls = ':', color="deeppink")
plt.ylim(1,2.5)
plt.xlim(0,np.max(delta_lambda_list)+np.min(delta_lambda_list))
plt.gca().set_aspect((((np.max(delta_lambda_list)+np.min(delta_lambda_list)))/1.5))
plt.yticks(np.arange(1,2.6,0.25))
plt.ylabel("Lifetime (ns)")
plt.xlabel("Delta Lambda (nm)")
plt.minorticks_on()
plt.show()

# Calculate the average bx_count_list per run time
bx_count_avg = []
x_count_avg = []
darkscounted_avg = []

# Number of measurements per run time
measurements_per_run_time = repeats 

# Iterate over the run_time_list
for i in range(len(delta_lambda_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time
    
    avg_bx_count = np.mean(bx_count_list[start_index:end_index])
    bx_count_avg.append(avg_bx_count)
    
    avg_x_count = np.mean(x_count_list[start_index:end_index])
    x_count_avg.append(avg_x_count)
    
    avg_darkscounted = np.mean(darkscounted[start_index:end_index])
    darkscounted_avg.append(avg_darkscounted)

# Print the average bx_count_list, x_count_list, and darkscounted per run time
print("Average BX Count per Run Time:", bx_count_avg)
print("Average X Count per Run Time:", x_count_avg)
print("Average Dark Counts per Run Time:", darkscounted_avg)

#Results SPAD heralded
print(run_time_list)
print(bx_decay_err_mean)
print(bx_decay_fit_mean)
print(x_decay_err_mean)
print(x_decay_fit_mean)
print(bx_spec_err_mean)
print(bx_spec_fit_mean)
print(x_spec_err_mean)
print(x_spec_fit_mean)


In [ ]:
#Results SPAD Voltages
bx_spec_fit_mean, bx_spec_err_mean = [], []
x_spec_fit_mean, x_spec_err_mean = [], []
x_decay_fit_mean, x_decay_err_mean = [], []
bx_decay_fit_mean, bx_decay_err_mean = [], []
bx_spec_std, x_spec_std, x_decay_std, bx_decay_std = [], [], [], []
bx_decay_volt = np.array(x_decay_volt)

# Number of measurements per run time
measurements_per_run_time = repeats

# Iterate over the run_time_list
for i in range(len(multiplier_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time

    x_spec_std.append(np.std(x_spec_fit[start_index:end_index]))
    bx_spec_std.append(np.std(bx_spec_fit[start_index:end_index]))
    x_decay_std.append(np.mean(x_decay_fit[start_index:end_index]))
    bx_decay_std.append(np.mean(bx_decay_fit[start_index:end_index]))

print(x_spec_std)
print(bx_spec_std)
print(x_decay_std)
print(bx_decay_std)
# Mean results for different voltages with mean error
volt_list = [5, 7, 9]

bx_spec_fit_mean, bx_spec_err_mean = [], []

for i in range(len(multiplier_list)):
    volt = multiplier_list[i]
    mask = []
    for j in range(len(bx_spec_volt)):
        mask.append(bx_spec_volt[j] == volt)   
    bx_spec_fit_mean.append(np.average(bx_spec_fit, weights=mask))
    bx_spec_err_mean.append(np.average(bx_spec_err, weights=mask))
plt.figure(dpi=300)
plt.errorbar(volt_list, bx_spec_fit_mean, bx_spec_err_mean, fmt='o', capsize=4, color='royalblue', label="Fitted Peak BX", markersize=5)
if mean_x < mean_bx:
    plt.ylim(mean_x - 1, mean_bx + 1)
else:
    plt.ylim(mean_bx - 1, mean_x + 1)


x_spec_fit_mean, x_spec_err_mean = [], []

for i in range(len(multiplier_list)):
    volt = multiplier_list[i]
    mask = []
    for j in range(len(x_spec_volt)):
        mask.append(x_spec_volt[j] == volt)   
    x_spec_fit_mean.append(np.average(x_spec_fit, weights=mask))
    x_spec_err_mean.append(np.average(x_spec_err, weights=mask))
plt.errorbar(volt_list, x_spec_fit_mean, x_spec_err_mean, fmt='o', capsize=4, color='crimson', label="Fitted Peak X", markersize=5)
plt.errorbar((0, 930), (mean_x, mean_x), ls=':', color="deeppink", label="Input Peak X")
plt.errorbar((0, 930), (mean_bx, mean_bx), ls=':', color="dodgerblue", label="Input Peak BX")
plt.gca().set_aspect(5/6)
plt.xlim(4.5, 9.5)
if mean_x < mean_bx:
    plt.ylim(mean_x - 1, mean_bx + 1)
else:
    plt.ylim(mean_bx - 1, mean_x + 1)
plt.legend()
plt.ylabel("Wavelength (nm)")
plt.xlabel("Voltage (V)")
plt.minorticks_on()
plt.show()

x_decay_fit_mean, x_decay_err_mean = [], []

for i in range(len(multiplier_list)):
    volt = multiplier_list[i]
    mask = []
    for j in range(len(x_decay_volt)):
        mask.append(x_decay_volt[j] == volt)   
    x_decay_fit_mean.append(np.average(x_decay_fit, weights=mask))
    x_decay_err_mean.append(np.average(x_decay_err, weights=mask))
plt.figure(dpi=300)
plt.errorbar(volt_list, np.array(x_decay_fit_mean)/1e3, np.array(x_decay_err_mean)/1e3, fmt='o', capsize=4, color='crimson', label="Fitted Lifetime X", markersize=5)
plt.errorbar((0, 930), (mean_life_x/1e3, mean_life_x/1e3), ls=':', color="deeppink", label="Input Lifetime X")
plt.gca().set_aspect(5/2)
plt.xlim(4.5, 9.5)
plt.ylim(23, 25)
plt.legend()
plt.ylabel("Lifetime (ns)")
plt.xlabel("Voltage (V)")
plt.minorticks_on()
plt.show()

bx_decay_fit_mean, bx_decay_err_mean = [], []
bx_decay_volt = np.array(x_decay_time)

for i in range(len(multiplier_list)):
    volt = multiplier_list[i]
    mask = []
    for j in range(len(bx_decay_volt)):
        mask.append(bx_decay_volt[j] == volt)   
    bx_decay_fit_mean.append(np.average(bx_decay_fit, weights=mask))
    bx_decay_err_mean.append(np.average(bx_decay_err, weights=mask))
plt.figure(dpi=300)
plt.errorbar(volt_list, np.array(bx_decay_fit_mean)/1e3, np.array(bx_decay_err_mean)/1e3, fmt='o', capsize=4, color='royalblue', label="Fitted Lifetime BX", markersize=5)
plt.errorbar((0, 930), (mean_life_rd_bx/1e3, mean_life_rd_bx/1e3), ls=':', color="dodgerblue", label="Input Lifetime BX")
plt.gca().set_aspect(5/4)
plt.xlim(4.5, 9.5)
plt.ylim(3, 7)
plt.legend()
plt.ylabel("Lifetime (ns)")
plt.xlabel("Voltage (V)")
plt.minorticks_on()
plt.show()

# Calculate the average bx_count_list per run time
bx_count_avg = []
x_count_avg = []
darkscounted_avg = []

# Number of measurements per run time
measurements_per_run_time = repeats

# Iterate over the run_time_list
for i in range(len(multiplier_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time
    
    avg_bx_count = np.mean(bx_count_list[start_index:end_index])
    bx_count_avg.append(avg_bx_count)
    
    avg_x_count = np.mean(x_count_list[start_index:end_index])
    x_count_avg.append(avg_x_count)
    
    avg_darkscounted = np.mean(darkscounted[start_index:end_index])
    darkscounted_avg.append(avg_darkscounted)

# Print the average bx_count_list, x_count_list, and darkscounted per run time
print("Average BX Count per Run Time:", bx_count_avg)
print("Average X Count per Run Time:", x_count_avg)
print("Average Dark Counts per Run Time:", darkscounted_avg)

#Results SPAD heralded
print(run_time_list)
print(bx_decay_err_mean)
print(bx_decay_fit_mean)
print(x_decay_err_mean)
print(x_decay_fit_mean)
print(bx_spec_err_mean)
print(bx_spec_fit_mean)
print(x_spec_err_mean)
print(x_spec_fit_mean)

In [ ]:
#Results SPAD Run times
bx_spec_fit_mean, bx_spec_err_mean = [], []
x_spec_fit_mean, x_spec_err_mean = [], []
x_decay_fit_mean, x_decay_err_mean = [], []
bx_decay_fit_mean, bx_decay_err_mean = [], []
bx_spec_std, x_spec_std, x_decay_std, bx_decay_std = [], [], [], []


# Number of measurements per run time
measurements_per_run_time = repeats

# Iterate over the run_time_list
for i in range(len(run_time_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time

    x_spec_std.append(np.std(x_spec_fit[start_index:end_index]))
    bx_spec_std.append(np.std(bx_spec_fit[start_index:end_index]))
    x_decay_std.append(np.mean(x_decay_fit[start_index:end_index]))
    bx_decay_std.append(np.mean(bx_decay_fit[start_index:end_index]))

print(x_spec_std)
print(bx_spec_std)
print(x_decay_std)
print(bx_decay_std)
# Mean results for different voltages with mean error
volt_list = run_time_list

bx_spec_fit_mean, bx_spec_err_mean = [], []

for i in range(len(run_time_list)):
    volt = run_time_list[i]
    mask = []
    for j in range(len(bx_spec_time)):
        mask.append(bx_spec_time[j] == volt)   
    bx_spec_fit_mean.append(np.average(bx_spec_fit, weights=mask))
    bx_spec_err_mean.append(np.average(bx_spec_err, weights=mask))
plt.figure(dpi=300)
plt.errorbar(volt_list, bx_spec_fit_mean, bx_spec_err_mean, fmt='o', capsize=4, color='royalblue', label="Fitted Peak BX", markersize=5)
if mean_x < mean_bx:
    plt.ylim(mean_x - 1, mean_bx + 1)
else:
    plt.ylim(mean_bx - 1, mean_x + 1)


x_spec_fit_mean, x_spec_err_mean = [], []

for i in range(len(run_time_list)):
    volt = run_time_list[i]
    mask = []
    for j in range(len(x_spec_time)):
        mask.append(x_spec_time[j] == volt)   
    x_spec_fit_mean.append(np.average(x_spec_fit, weights=mask))
    x_spec_err_mean.append(np.average(x_spec_err, weights=mask))
plt.errorbar(volt_list, x_spec_fit_mean, x_spec_err_mean, fmt='o', capsize=4, color='crimson', label="Fitted Peak X", markersize=5)
plt.errorbar((0, 930), (mean_x, mean_x), ls=':', color="deeppink", label="Input Peak X")
plt.errorbar((0, 930), (mean_bx, mean_bx), ls=':', color="dodgerblue", label="Input Peak BX")
plt.gca().set_aspect(930/6)
plt.xlim(0, 930)
if mean_x < mean_bx:
    plt.ylim(mean_x - 1, mean_bx + 1)
else:
    plt.ylim(mean_bx - 1, mean_x + 1)
plt.legend()
plt.ylabel("Wavelength (nm)")
plt.xlabel("Run Time (s)")
plt.minorticks_on()
plt.show()

x_decay_fit_mean, x_decay_err_mean = [], []

for i in range(len(run_time_list)):
    volt = run_time_list[i]
    mask = []
    for j in range(len(x_decay_time)):
        mask.append(x_decay_time[j] == volt)   
    x_decay_fit_mean.append(np.average(x_decay_fit, weights=mask))
    x_decay_err_mean.append(np.average(x_decay_err, weights=mask))
plt.figure(dpi=300)
plt.errorbar(volt_list, np.array(x_decay_fit_mean)/1e3, np.array(x_decay_err_mean)/1e3, fmt='o', capsize=4, color='crimson', label="Fitted Lifetime X", markersize=5)
plt.errorbar((0, 930), (mean_life_x/1e3, mean_life_x/1e3), ls=':', color="deeppink", label="Input Lifetime X")
plt.gca().set_aspect(930/2)
plt.xlim(0, 930)
plt.ylim(23, 25)
plt.legend()
plt.ylabel("Lifetime (ns)")
plt.xlabel("Run Time (s)")
plt.minorticks_on()
plt.show()

bx_decay_fit_mean, bx_decay_err_mean = [], []
bx_decay_volt = np.array(x_decay_time)

for i in range(len(run_time_list)):
    volt = run_time_list[i]
    mask = []
    for j in range(len(bx_decay_volt)):
        mask.append(bx_decay_volt[j] == volt)   
    bx_decay_fit_mean.append(np.average(bx_decay_fit, weights=mask))
    bx_decay_err_mean.append(np.average(bx_decay_err, weights=mask))
plt.figure(dpi=300)
plt.errorbar(volt_list, np.array(bx_decay_fit_mean)/1e3, np.array(bx_decay_err_mean)/1e3, fmt='o', capsize=4, color='royalblue', label="Fitted Lifetime BX", markersize=5)
plt.errorbar((0, 930), (mean_life_rd_bx/1e3, mean_life_rd_bx/1e3), ls=':', color="dodgerblue", label="Input Lifetime BX")
plt.gca().set_aspect(930/4)
plt.xlim(0, 930)
plt.ylim(3, 7)
plt.legend()
plt.ylabel("Lifetime (ns)")
plt.xlabel("Run Time (s)")
plt.minorticks_on()
plt.show()

# Calculate the average bx_count_list per run time
bx_count_avg = []
x_count_avg = []
darkscounted_avg = []

# Number of measurements per run time
measurements_per_run_time = 10

# Iterate over the run_time_list
for i in range(len(run_time_list)):
    start_index = i * measurements_per_run_time
    end_index = start_index + measurements_per_run_time
    
    avg_bx_count = np.mean(bx_count_list[start_index:end_index])
    bx_count_avg.append(avg_bx_count)
    
    avg_x_count = np.mean(x_count_list[start_index:end_index])
    x_count_avg.append(avg_x_count)
    
    avg_darkscounted = np.mean(darkscounted[start_index:end_index])
    darkscounted_avg.append(avg_darkscounted)

# Print the average bx_count_list, x_count_list, and darkscounted per run time
print("Average BX Count per Run Time:", bx_count_avg)
print("Average X Count per Run Time:", x_count_avg)
print("Average Dark Counts per Run Time:", darkscounted_avg)

#Results SPAD heralded
print(run_time_list)
print(bx_decay_err_mean)
print(bx_decay_fit_mean)
print(x_decay_err_mean)
print(x_decay_fit_mean)
print(bx_spec_err_mean)
print(bx_spec_fit_mean)
print(x_spec_err_mean)
print(x_spec_fit_mean)